In [10]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
import time

In [4]:
def save(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, package=row[0], count=row[1])))
        df.show(5)
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="cran_counts", keyspace="streamingbdm")\
        .save(mode="append")
    except:
        pass

========= 2021-12-22 04:01:18 =========


In [5]:
ssc = StreamingContext(sc, 3)
cran_data = ssc.textFileStream("file:///home/victorbdm/assignStreaming/")
cran_data = cran_data.map(lambda x: x.split(','))

def remove_quotation(x):
    return([xx.replace('"', '') for xx in x])
cran_data = cran_data.map(remove_quotation)

package_download_count = cran_data.map(lambda package:(package[6], 1))
package_download_count = package_download_count.reduceByKey(lambda a,b: a+b)
package_download_count.pprint()
package_download_count.foreachRDD(save)
ssc.start()
time.sleep(120)
ssc.stop(stopSparkContext=False)

Py4JJavaError: An error occurred while calling o12382.start.
: java.lang.IllegalStateException: Only one StreamingContext may be started in this JVM. Currently running StreamingContext was started atorg.apache.spark.streaming.api.java.JavaStreamingContext.start(JavaStreamingContext.scala:557)
sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
java.lang.reflect.Method.invoke(Method.java:498)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:282)
py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
py4j.commands.CallCommand.execute(CallCommand.java:79)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerConnection.run(ClientServerConnection.java:106)
j

========= 2021-12-22 04:01:21 =========
========= 2021-12-22 04:01:24 =========
========= 2021-12-22 04:01:27 =========
========= 2021-12-22 04:01:30 =========
========= 2021-12-22 04:01:33 =========
========= 2021-12-22 04:01:36 =========
========= 2021-12-22 04:01:39 =========
========= 2021-12-22 04:01:42 =========
========= 2021-12-22 04:01:45 =========
========= 2021-12-22 04:01:48 =========
========= 2021-12-22 04:01:51 =========
========= 2021-12-22 04:01:54 =========
========= 2021-12-22 04:01:57 =========
========= 2021-12-22 04:02:00 =========
========= 2021-12-22 04:02:03 =========
========= 2021-12-22 04:02:06 =========
========= 2021-12-22 04:02:09 =========
========= 2021-12-22 04:02:12 =========
========= 2021-12-22 04:02:15 =========
========= 2021-12-22 04:02:18 =========
========= 2021-12-22 04:02:21 =========
========= 2021-12-22 04:02:24 =========
========= 2021-12-22 04:02:27 =========
========= 2021-12-22 04:02:30 =========
========= 2021-12-22 04:02:33 =========


### question 2

In [22]:
def save(time, rdd):
    
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(timenew=time, date=row[0], time=row[1], size=row[2], r_version=row[3], r_arch=row[4], r_os=row[5], package=row[6], version=row[7], country=row[8], ip_id=row[9])))
        df.createOrReplaceTempView('cran_data')
        df = spark.sql('SELECT package, MAX(package) AS package_count FROM cran_data GROUP BY package ORDER BY package_count DESC')
        df.show()
        df.select("package", "package_count")\
        .write.format("org.apache.spark.sql.cassandra")\
        .options(table="top_package", keyspace="streamingbdm")\
        .save(mode="append")
    except:
        pass

In [23]:
ssc = StreamingContext(sc, 3)

cran_data = ssc.textFileStream("file:///home/victorbdm/assignStreaming/")
cran_data = cran_data.map(lambda x: x.split(','))

def remove_quotation(x):
    return([xx.replace('"', '') for xx in x])
cran_data = cran_data.map(remove_quotation)
cran_data.foreachRDD(save)
ssc.start()
time.sleep(120)
ssc.stop(stopSparkContext=False)

+------------+-------------+
|     package|package_count|
+------------+-------------+
|         zoo|          zoo|
|         zip|          zip|
|     zeallot|      zeallot|
|   yardstick|    yardstick|
|        yaml|         yaml|
|         xts|          xts|
|      xtable|       xtable|
|        xml2|         xml2|
|    xlsxjars|     xlsxjars|
|        xlsx|         xlsx|
|        xfun|         xfun|
|    xaringan|     xaringan|
|      worrms|       worrms|
|workflowsets| workflowsets|
|   workflows|    workflows|
|          wk|           wk|
|       withr|        withr|
|    wikitaxa|     wikitaxa|
|     whisker|      whisker|
|     webshot|      webshot|
+------------+-------------+
only showing top 20 rows

+-----------+-------------+
|    package|package_count|
+-----------+-------------+
|        zoo|          zoo|
|        zip|          zip|
|       yaml|         yaml|
|        xts|          xts|
|     xtable|       xtable|
|       xml2|         xml2|
|       xfun|         xfun

### question 3

In [11]:
def save(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(timenew=time, date=row[0], time=row[1], size=row[2], r_version=row[3], r_arch=row[4], r_os=row[5], package=row[6], version=row[7], country=row[8], ip_id=row[9])))
        df.createOrReplaceTempView('cran_data')
        df = spark.sql('SELECT country, COUNT(package) AS download_count FROM cran_data GROUP BY country ORDER BY download_count DESC')
        df.show(5)
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="country_counts", keyspace="streamingbdm")\
        .save(mode="append")
    except:
        pass

In [12]:

ssc = StreamingContext(sc, 3)
cran_data = ssc.textFileStream("file:///home/victorbdm/assignStreaming/")
cran_data = cran_data.map(lambda x: x.split(','))

def remove_quotation(x):
    return([xx.replace('"', '') for xx in x])
cran_data = cran_data.map(remove_quotation)
cran_data.foreachRDD(save)
ssc.start()
time.sleep(120)
ssc.stop(stopSparkContext=False)

+-------+--------------+
|country|download_count|
+-------+--------------+
|     US|           436|
|     GB|           193|
|     NA|           125|
|     ES|            36|
|     BR|            23|
+-------+--------------+
only showing top 5 rows

+-------+--------------+
|country|download_count|
+-------+--------------+
|     US|           457|
|     GB|           195|
|     NA|           135|
|     PL|            27|
|     AE|            26|
+-------+--------------+
only showing top 5 rows

+-------+--------------+
|country|download_count|
+-------+--------------+
|     US|           955|
|     GB|           364|
|     NA|           211|
|     CL|            50|
|     ES|            40|
+-------+--------------+
only showing top 5 rows

+-------+--------------+
|country|download_count|
+-------+--------------+
|     US|           505|
|     GB|           147|
|     NA|            99|
|     HK|            36|
|     ES|            22|
+-------+--------------+
only showing top 5 rows



### question 4

In [2]:
def save(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(\
        lambda row: Row(time=time, package=row[0], gp_count=row[1])))
        df.show(5)
        df.write.format("org.apache.spark.sql.cassandra")\
        .options(table="gplot_counts", keyspace="streamingbdm")\
        .save(mode="append")
    except:
        pass

In [3]:
ssc = StreamingContext(sc, 3)
cran_data = ssc.textFileStream("file:///home/victorbdm/assignStreaming/")
cran_data = cran_data.map(lambda x: x.split(','))

def remove_quotation(x):
    return([xx.replace('"', '') for xx in x])
cran_data = cran_data.map(remove_quotation)
ggplot2_package = cran_data.map(lambda package:(package[6], 1))
ggplot2_package = ggplot2_package.reduceByKey(lambda a,b: a+b)
ggplot2_package= ggplot2_package.filter(lambda a: 'ggplot2' in a)
ggplot2_package.foreachRDD(save)

ssc.start()
time.sleep(120)
ssc.stop(stopSparkContext=False)

+-------------------+-------+--------+
|               time|package|gp_count|
+-------------------+-------+--------+
|2021-12-22 02:17:09|ggplot2|      54|
+-------------------+-------+--------+

+-------------------+-------+--------+
|               time|package|gp_count|
+-------------------+-------+--------+
|2021-12-22 02:17:15|ggplot2|      34|
+-------------------+-------+--------+

+-------------------+-------+--------+
|               time|package|gp_count|
+-------------------+-------+--------+
|2021-12-22 02:17:18|ggplot2|      22|
+-------------------+-------+--------+

+-------------------+-------+--------+
|               time|package|gp_count|
+-------------------+-------+--------+
|2021-12-22 02:17:21|ggplot2|      17|
+-------------------+-------+--------+

+-------------------+-------+--------+
|               time|package|gp_count|
+-------------------+-------+--------+
|2021-12-22 02:17:24|ggplot2|      15|
+-------------------+-------+--------+

+-------------------